# Лабораторная работа 2. Исследование латентного пространства VAE. Исследование codebook VQ-VAE

Данная лабораторная работа состоит из двух частей: исследование латентного пространства VAE и исследование codebook VQ-VAE.

Предлагается обучить VAE и VQ-VAE на датасете fashionMNIST. Лабораторная работа должна быть выполнена на Pytorch.

1. Что такое VAE? Зачем его придумали? Чем он отличается от обычного автокодировщика?
2. Какие проблемы решает VQ-VAE? Чем он отличается от автокодировщика и вариационного автокодировщика?
3. Что такое квантование в VQ-VAE?

## Подгрузка импортов и датасета

In [ ]:
import os
import os.path as osp
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as func
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [ ]:
# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
])

# Load the dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

In [ ]:
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

## Часть 1. Исследование латентного пространства VAE

В этой части лабораторной работы предлагается обучить обычный VAE. С помощью инструментов снижения размерности (t-SNE или PCA) визуализируйте на плоскости внутреннее пространство VAE.
Альтернативой станет выбор `dim_code=2` и визуализация результатов этого вариационного кодировщика.

In [ ]:
class VAE(nn.Module):
    def __init__(self, dim_code):
        super().__init__()

        self.enc = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Flatten()
        )

        self.mu = ...
        self.log_var = ...

        self.decoder_input = ...

        self.dec = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3,  stride=2),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.ConvTranspose2d(in_channels=32, out_channels=1, kernel_size=3,  stride=2, output_padding=1),
            nn.LeakyReLU(),
            nn.Sigmoid()
        )


    def encode(self, x):
        pass

    def gaussian_sampler(self, mu, log_var):
        pass

    def decode(self, z):
        pass

    def forward(self, x):
        pass

In [ ]:
class VAELoss(nn.Module):
    def __init__(self, kl_weight=1, mse_weight=1):
        pass
    
    def _kl_loss(self, mu, log_var):
        pass

    def forward(self, x, reconstruction, mu, log_var):
        pass

In [ ]:
def train_autoencoder(model, dataloader, criterion, optimizer, epochs, device):
    train_loss = []

    for i in range(epochs):
        model.to(device)
        model.train()
        train_epoch_loss = []
        tqdm_iter = tqdm(dataloader)
        for batch in tqdm_iter:
            images, _ = batch
            images = images.to(device)

            optimizer.zero_grad()

            pred = model(images)
            reconstructed, mu, log_var = pred
            loss = criterion(images, reconstructed, mu, log_var)
            
            loss.backward()
            optimizer.step()
            tqdm_iter.set_postfix(loss=f'{loss.item():.3f}')
        train_epoch_loss.append(loss.item())
        train_loss.append(np.mean(train_epoch_loss))
        tqdm_iter.set_postfix(loss=f'{train_loss[-1]:.5f}')
    return train_loss

In [ ]:
criterion = ...
model = ...
optimizer = ...
epochs = ...
device = ...

In [ ]:
loss = train_autoencoder(
    model=model,
    dataloader=dataloader,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs,
    device=device
)

plt.figure(figsize=(12, 4))
plt.plot(loss)
plt.show()

In [ ]:
def vizualise_latent_space(
        model,
        # <your args here>
):
    pass

## Часть 2. Исследование codebook VQ-VAE

Исследуйте влияние гиперпараметров `embedding_dim, num_embeddings` на генерацию и сходимость VQ-VAE. В пределах каких значений генерация лучше? В пределах каких значений сходится кодировщик? От чего зависит диапазон значений для модели?

In [ ]:
class VectorQuantizer(nn.Module):
    def __init__(self):
        pass
    
    def forward(self, x):
        pass

In [ ]:
class QuantizerLoss(nn.Module):
    def __init__(self, commitment_cost):
        super().__init__()
        self.commitment_cost = commitment_cost
        self.mse = nn.MSELoss()
    
    def forward(self, z, quantized):
        return self.mse(quantized.detach(), z) + self.commitment_cost * self.mse(quantized, z.detach())

In [ ]:
class VQVAELoss(nn.Module):
    def __init__(self, commitment_cost=0.5):
        super().__init__()
        self.recon = nn.MSELoss()
        self.quantizer = QuantizerLoss(commitment_cost)
    
    def forward(self, x, recon, z, quantized):
        return self.recon(x, recon) + self.quantizer(z, quantized)

In [ ]:
class VQVAE(nn.Module):
    def __init__(
            self,
            num_embeddings=10,
            embedding_dim=128,
            ):
        super().__init__()

        self.enc = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2),
            nn.LeakyReLU(),
        )

        self.quantizer = ...

        self.dec = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3,  stride=2),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.ConvTranspose2d(in_channels=32, out_channels=1, kernel_size=3,  stride=2, output_padding=1),
            nn.LeakyReLU(),
            nn.Sigmoid()
        )


    def encode(self, x):
        pass

    def decode(self, z):
        pass

    def forward(self, x):
        pass

### Исследование влияния гиперпараметров на результаты обучения

In [ ]:
criterion = ...
model = ...
optimizer = ...
epochs = ...
device = ...